In [1]:
from numpy import linspace, zeros, cumsum, sqrt, ones, where, mean, std, unique, correlate, corrcoef
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import t
import csv
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
def readLines(data_file):
    r = csv.reader(open(data_file, "rt", newline=''), dialect="excel")
    return [l for l in r]

Couple notes: To run this code, change the directory to one you use. Second, the first "filename" is the noise complaint data, which you should have. The other files are the crime data, which I should have attached in the email this came in.

In [3]:
directory = "e:/Franco/Documents/Schoolwork/College/Masters/CSC440/FinalProjectData/"

filename = "noise_complaint_data_2010-2015.csv"

noise_data = readLines(directory+filename)


In [84]:
directory2 = directory+'historical_citywide_crime_complaint_data_by_precinct_2000_2015/'
file_misd = "misdemeanor_offenses_by_precinct_2000_2015.csv"
file_major_fel = "seven_major_felony_offenses_2000_2015.csv"
file_nonmajor_fel = "non_seven_major_felony-offenses_by_precinct_2000_2015.csv"
file_violation = "violation_offenses_by_precinct_2000_2015.csv"

misd_data = readLines(directory2+file_misd)
major_fels = readLines(directory2+file_major_fel)
nonmajor_fels = readLines(directory2+file_nonmajor_fel)
violat_data = readLines(directory2+file_violation)

The following code preprocesses the crime data.

In [85]:
misd_data.pop(0)
misd_data.pop(0)
misd_atts = misd_data[0][:]
misd_data.pop(0)

major_fels.pop(0)
major_fels.pop(0)
major_fels_atts = major_fels[0][:]
major_fels.pop(0)

nonmajor_fels.pop(0)
nonmajor_fels.pop(0)
nonmajor_fels_atts = nonmajor_fels[0][:]
nonmajor_fels.pop(0)

violat_data.pop(0)
violat_data.pop(0)
violat_atts = violat_data[0][:]
violat_data.pop(0)

['PCT',
 'CRIME',
 '2000',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015']

In [86]:
num_misd = 18

for i in range(0,len(misd_data)):
    precinct = int(i/num_misd)*num_misd
    misd_data[i][0] = misd_data[precinct][0]
for i in range(0,len(misd_data)-1386):
    misd_data.pop()
    
num_major_fels = 8
for i in range(0,len(major_fels)):
    precinct = int(i/num_major_fels)*num_major_fels
    major_fels[i][0] = major_fels[precinct][0]
for i in range(0,len(major_fels)-616):
    major_fels.pop()
    
num_nonmajor_fels = 9
for i in range(0,len(nonmajor_fels)):
    precinct = int(i/num_nonmajor_fels)*num_nonmajor_fels
    nonmajor_fels[i][0] = nonmajor_fels[precinct][0]
for i in range(0,len(nonmajor_fels)-693):
    nonmajor_fels.pop()
    
num_violat_data = 3
for i in range(0,len(violat_data)):
    precinct = int(i/num_violat_data)*num_violat_data
    violat_data[i][0] = violat_data[precinct][0]
for i in range(0,len(violat_data)-231):
    violat_data.pop()

In [246]:
def transpose_matrix(matrix):
    outmatrix = zeros([len(matrix[0]),len(matrix)])
    for x in range(0,len(matrix[0])):
        for y in range(0,len(matrix)):
            outmatrix[x][y] = matrix[y][x]
    return outmatrix
    #height = len(matrix)
    #width = len(matrix[0])
    #return [[ matrix[row][col] for col in range(0,width) ] for row in range(0,height) ]

In [274]:
def nominalize_data(prefix,matrix,years):
    outmatrix = [[None]*len(matrix[0])]*len(years)
    median1 = zeros(len(years))
    std1 = zeros(len(years))
    for y in years:
        y1 = y-min(years)
        median1[y1] = median(matrix[years][:])
        std1[y1] = np.std(matrix[years][:])
        for x in range(0,len(matrix[y])):
            if matrix[y][x] > median1[y1]+3*std1[y1]:
                outmatrix[y1][x] = prefix+": Substantially Higher"
            elif matrix[y][x] > median1[y1]+2*std1[y1]:
                outmatrix[y1][x] = prefix+": Much Higher"
            elif matrix[y][x] > median1[y1]+std1[y1]:
                outmatrix[y1][x] = prefix+": Slightly Higher"
            if matrix[y][x] < median1[y1]-3*std1[y1]:
                outmatrix[y1][x] = prefix+": Substantially Lower"
            elif matrix[y][x] < median1[y1]-2*std1[y1]:
                outmatrix[y1][x] = prefix+": Much Lower"
            elif matrix[y][x] < median1[y1]-std1[y1]:
                outmatrix[y1][x] = prefix+": Slightly Lower"
            elif matrix[y][x] >= median1[y1]-std1[y1] and matrix[y][x] <= median1[y1]+std1[y1]:
                outmatrix[y1][x] = prefix+": Approx. Median"
    print(median1,std1)
    return outmatrix

##NOTE: I only discretized the data from 2013-2015.
###This is because Precinct 121 only has crime data from 2013 onwards. If you know of a way to get around this, tell me.

##Discretize Major Felonies Data

In [275]:
years = 16
num_precincts = int(len(major_fels)/num_major_fels)
sum_major_fels = zeros([num_precincts,years])
for i in range(0,num_precincts):
    prec_index = (i+1)*num_major_fels-1
    for j in range(years-3,years):
        #print(misd_data[prec_index][j+2])
        sum_major_fels[i][j] = int(major_fels[prec_index][j+2])
sum_major_fels1 = transpose_matrix(sum_major_fels)
years = [13,14,15]
prefix = "MAJOR FELONIES"
nominal_major_fels = nominalize_data(prefix,sum_major_fels1,years)

##Discretize Nonmajor Felonies Data

In [282]:
years = 16
num_precincts = int(len(nonmajor_fels)/num_nonmajor_fels)
sum_nonmajor_fels = zeros([num_precincts,years])
for i in range(0,num_precincts):
    prec_index = (i+1)*num_nonmajor_fels-1
    for j in range(years-3,years):
        #print(misd_data[prec_index][j+2])
        sum_nonmajor_fels[i][j] = int(nonmajor_fels[prec_index][j+2])
sum_nonmajor_fels1 = transpose_matrix(sum_nonmajor_fels)
years = [13,14,15]
prefix = "NONMAJOR FELONIES"
nominal_nonmajor_fels = nominalize_data(prefix,sum_nonmajor_fels1,years)

[ 664.  664.  664.] [ 413.92857237  413.92857237  413.92857237]


##Discretize Violation Offenses Data

In [291]:
years = 16
num_precincts = int(len(violat_data)/num_violat_data)
sum_violat = zeros([num_precincts,years])
for i in range(0,num_precincts):
    prec_index = (i+1)*num_violat_data-1
    for j in range(years-3,years):
        #print(misd_data[prec_index][j+2])
        sum_violat[i][j] = int(violat_data[prec_index][j+2])
sum_violat1 = transpose_matrix(sum_violat)
years = [13,14,15]
prefix = "VIOLATION OFFENSES"
nominal_violat = nominalize_data(prefix,sum_violat1,years)

[ 784.  784.  784.] [ 363.4916373  363.4916373  363.4916373]


##Discretize Misdemeanor Data

In [293]:
years = 16
num_precincts = int(len(misd_data)/num_misd)
sum_misd = zeros([num_precincts,years])
for i in range(0,num_precincts):
    prec_index = (i+1)*num_misd-1
    for j in range(years-3,years):
        #print(misd_data[prec_index][j+2])
        sum_misd[i][j] = int(misd_data[prec_index][j+2])
sum_misd1 = transpose_matrix(sum_misd)
years = [13,14,15]
prefix = "MISDEMEANOR"
nominal_misd = nominalize_data(prefix,sum_misd1,years)

[ 4089.  4089.  4089.] [ 2117.61356848  2117.61356848  2117.61356848]


##FORMAT OF THE FILES
###Rows are years (2013, 2014, 2015)
###Columns are precincts

In [298]:
with open('nom_misd.csv', 'w', newline='\n') as csvfile:
    filewriter = csv.writer(csvfile)
    filewriter.writerows(nominal_misd)
with open('nom_major_fels.csv', 'w', newline='\n') as csvfile:
    filewriter = csv.writer(csvfile)
    filewriter.writerows(nominal_major_fels)
with open('nom_nonmajor_fels.csv', 'w', newline='\n') as csvfile:
    filewriter = csv.writer(csvfile)
    filewriter.writerows(nominal_nonmajor_fels)
with open('nom_violat.csv', 'w', newline='\n') as csvfile:
    filewriter = csv.writer(csvfile)
    filewriter.writerows(nominal_violat)

In [273]:
len(major_fels)

616

In [247]:
sum_misd1 = transpose_matrix(sum_misd)

In [248]:
sum_misd1[13]

array([  4503.,   3437.,   4035.,   2970.,   4290.,   3077.,   5801.,
        10483.,   2075.,   5005.,   3527.,   2636.,    345.,   4930.,
         3165.,   7000.,   2753.,   5074.,   4089.,   4753.,   4481.,
         4480.,  10544.,   5482.,   6078.,   8795.,  10609.,   3580.,
         7316.,   6019.,   6572.,   5348.,   2547.,   8143.,   4441.,
         3355.,   3240.,   2763.,   2758.,   5928.,   3205.,   3019.,
         5310.,   4441.,   3697.,   7877.,  12510.,   1845.,   5397.,
         2783.,   6236.,   4808.,   5195.,   3771.,   2810.,   4857.,
         2357.,   1494.,   3134.,   4131.,   6521.,   4989.,   4178.,
         3906.,   3482.,   3594.,   5094.,   5142.,   1637.,   2574.,
         5608.,   6188.,   5977.,   6462.,   2325.,   4406.,   1963.])

In [249]:
sum_misd

array([[    0.,     0.,     0., ...,  4503.,  4043.,  3881.],
       [    0.,     0.,     0., ...,  3437.,  3401.,  3339.],
       [    0.,     0.,     0., ...,  4035.,  3718.,  3397.],
       ..., 
       [    0.,     0.,     0., ...,  2325.,  5042.,  4414.],
       [    0.,     0.,     0., ...,  4406.,  3353.,  2960.],
       [    0.,     0.,     0., ...,  1963.,  1888.,  1721.]])

[ 4089.  4089.  4089.] [ 2117.61356848  2117.61356848  2117.61356848]
